# Implementación de TICA: datos de E=-0,14 VSD_4ms2

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import re
from pyemma.coordinates import tica
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import StandardScaler

## Carga de datos

In [ ]:
path = "/home/lcampos/zeus/proyecto_fondecyt/CODE/"
df_energy = pd.read_csv(path+'csv_creados/df_energy_E-014_VSD_4ms2.csv')
df_energy.set_index(['R', 'Unnamed: 1'], inplace=True) 
df_energy.rename_axis(index={'Unnamed: 1': 'Time'}, inplace=True)
df_pca = pd.read_csv(path+'csv_creados/df_pca_E-014_VSD_4ms2.csv')
df_pca.set_index(['R', 'Unnamed: 1'], inplace=True)  
df_pca.rename_axis(index={'Unnamed: 1': 'Time'}, inplace=True)

In [ ]:
data = df_energy.join([df_pca])
data

## Funciones para implementar TICA

A diferencia de UMAP, TICA requiere los datos normalizados de manera que tengan media $0$ y std $1$.

Con dim=-1 TICA elige el valor de dim (tomando los valores propios que esten sobre cierto umbral). Para efectos de la implementación, siempre se usará dim=-1 pues TICA comunmente (en este set de datos) retorna mas de 2 dimensiones

In [ ]:
def implementacion_TICA(dataset, lag=50, dim=-1):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(dataset.values)
    p_t = tica(X_scaled, lag=lag, dim=dim)
    embedding = p_t.get_output()[0]
    
    return p_t, embedding

def grafico_TICA(embedding, color, dim=2, title='TICA Projection', s=5):
    if dim == 2:
        plt.figure(figsize=(8, 6))
        plt.scatter(embedding[:, 0], embedding[:, 1], c = color, s=s)
        plt.title(title)
        plt.xlabel('Dimension 1')
        plt.ylabel('Dimension 2')
        plt.show() 
        
    elif dim == 3:
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(embedding[:, 0], embedding[:, 1], embedding[:, 2], c=color, cmap='viridis', s=s)
        ax.set_title(title)
        ax.set_xlabel('Dimension 1')
        ax.set_ylabel('Dimension 2')
        ax.set_zlabel('Dimension 3')
        plt.show()

In [ ]:
# La siguiente función sirve para ver los dimensiones más ""significativas"" (en el sentido explicado 
# en el informe/presentacion). Nota: Funciona solo para datasets que tengan mas de una repetición

def dims_significativas(dataset, rep=100, lag=50, dim=-1):
    imp_per_rep = {}  # dimensiones importantes por repeticion
    imp_all =[]  # todas las dimensiones importantes (se repiten por cada valor ppio de cada Simulación)
    imp = []  # Dims importantes
    for i in range(rep):
        df_i = dataset.loc[dataset.index.get_level_values('R') == i+1]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(df_i.values)
        p_t_i = tica(X_scaled, lag=50, dim=-1)
        embedding_i = p_t_i.get_output()[0]
        eig_vect_i = p_t_i.eigenvectors
        eig_val_i = p_t_i.eigenvalues
        imp_per_rep[i+1] = {}

        a = []
        for j in range(3):  # Solo considero los 3 valores ppios mas grandes
            modulos = np.abs(eig_vect_i[:, j])
            umbral = np.mean(modulos) + 1 * np.std(modulos)  # Este criterio puede cambiar
            indices_greater = np.where(np.abs(eig_vect_i[:,j]) > umbral)
            imp_per_rep[i+1][j] = [data.columns[x] for x in np.unique(indices_greater[0])]
            for x in np.unique(indices_greater[0]):
                imp_all.append(data.columns[x])
                if data.columns[x] not in a:
                    a.append(data.columns[x])
        for x in a:
            imp.append(x)
    # se podrían retornar también las demás, pero para efectos de lo que presentaré (y usaré) solo me interesa esta        
    return imp

## Tica a cada repetición

In [ ]:
# tau = 50
for i in range(100):
    df_i = data.loc[data.index.get_level_values('R') == i+1]
    p_t, embedding = implementacion_TICA(df_i, lag=50, dim=-1)
    grafico_TICA(embedding, np.linspace(0,50000,5001), dim=2, title='TICA Projection 2D- Repetition'+str(i+1), s=5)
    grafico_TICA(embedding, np.linspace(0,50000,5001), dim=3, title='TICA Projection 3D- Repetition'+str(i+1), s=5)

In [ ]:
# tau = 600
for i in range(100):
    df_i = data.loc[data.index.get_level_values('R') == i+1]
    p_t, embedding = implementacion_TICA(df_i, lag=600, dim=-1)
    grafico_TICA(embedding, np.linspace(0,50000,5001), dim=2, title='TICA Projection 2D- Repetition'+str(i+1), s=5)
    grafico_TICA(embedding, np.linspace(0,50000,5001), dim=3, title='TICA Projection 3D- Repetition'+str(i+1), s=5)

In [ ]:
# tau = 1000
for i in range(100):
    df_i = data.loc[data.index.get_level_values('R') == i+1]
    p_t, embedding = implementacion_TICA(df_i, lag=1000, dim=-1)
    grafico_TICA(embedding, np.linspace(0,50000,5001), dim=2, title='TICA Projection 2D- Repetition'+str(i+1), s=5)
    grafico_TICA(embedding, np.linspace(0,50000,5001), dim=3, title='TICA Projection 3D- Repetition'+str(i+1), s=5)

In [ ]:
imp = dims_significativas(data, rep=100, lag=50, dim=-1)

In [ ]:
frecuencia_elementos = Counter(imp)

for elemento, frecuencia in frecuencia_elementos.items():
    print(f"{elemento}: {frecuencia} veces")

In [ ]:
elementos, frecuencias = zip(*sorted(frecuencia_elementos.items(), key=lambda x: x[1], reverse=True))
plt.figure(figsize=(8, 6))
plt.plot(elementos, frecuencias, marker='o')
plt.xticks(rotation=60, ha='right')
plt.show()

In [ ]:
frecuencia_elementos = Counter(imp)
columnas_tica = []

for elemento, frecuencia in frecuencia_elementos.items():
    if frecuencia>40:
        print(f"{elemento}: {frecuencia} veces")
        columnas_tica.append(elemento)

## TICA sobre todo el set de datos

In [ ]:
tiempo = np.tile(np.linspace(0,50000,5001),100)
p_t, embedding = implementacion_TICA(data, lag=50, dim=-1) 
grafico_TICA(embedding, tiempo, dim=2, title='TICA Projection 2D all data', s=1)
grafico_TICA(embedding, tiempo, dim=3, title='TICA Projection 3D all data', s=1)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(embedding[:, 1], embedding[:, 0], embedding[:, 2], c=tiempo, cmap='viridis', s=1)
ax.set_title('TICA Projection 3D all data')
ax.set_xlabel('Dimension 2')
ax.set_ylabel('Dimension 1')
ax.set_zlabel('Dimension 3')
plt.show()

In [ ]:
for i in range(100):
    subset_embedding = embedding[5001*i:5001*(i+1)]  
    plt.figure(figsize=(8, 6))
    plt.scatter(subset_embedding[:, 0], subset_embedding[:, 1], c = np.linspace(0,50000,5001), s=5)
    plt.xlim([-2, 5])
    plt.ylim([-4, 4])
    plt.title('TICA projection 3D of all repetition, repetition '+str(i+1))
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.show()

In [ ]:
for i in range(100):
    subset_embedding = embedding[5001*i:5001*(i+1)]  
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(subset_embedding[:, 0], subset_embedding[:, 1], subset_embedding[:, 2], c=np.linspace(0,50000,5001), cmap='viridis', s=5)
    ax.set_title('TICA projection 3D of all repetition, repetition '+str(i+1))
    ax.set_xlabel('Dimension 1')
    ax.set_ylabel('Dimension 2')
    ax.set_zlabel('Dimension 3')
    plt.show()

In [ ]:
imp_all =[]
imp = []

eig_vect = p_t.eigenvectors
eig_val = p_t.eigenvalues

a = []
for j in range(3):
    modulos = np.abs(eig_vect[:, j])
    umbral = np.mean(modulos) + 1 * np.std(modulos)
    indices_greater = np.where(np.abs(eig_vect[:,j]) > umbral)
    #print(eig_vect[:,j])
    for x in np.unique(indices_greater[0]):
        imp_all.append(data.columns[x])
        if data.columns[x] not in a:
            a.append(data.columns[x])
for x in a:
    imp.append(x)

In [ ]:
imp

In [ ]:
df = data[columnas_tica]
tiempo = np.tile(np.linspace(0,50000,5001),100)
p_t, embedding = implementacion_TICA(df, lag=50, dim=-1) 
grafico_TICA(embedding, tiempo, dim=2, title='TICA Projection 2D all data', s=1)
grafico_TICA(embedding, tiempo, dim=3, title='TICA Projection 3D all data', s=1)